<a href="https://colab.research.google.com/github/KavyaGarapati/HOMEWORK-2/blob/master/k-fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D
import os

In [0]:
batch_size = 32
num_classes = 10
epochs = 100


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, y_train = shuffle(x_train, y_train, random_state = 0)
split_index = int(0.8 * x_train.shape[0])

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
  
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

#train_x = x_train[ :split_index]
#train_y = y_train[ :split_index]

#validate_x = x_train[ :split_index]
#validate_y = y_train[ :split_index]

#print("\no of samples in training set: ",len(train_x))
#print("\no of samples in validation set: ",len(validate_x))


In [0]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])



In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 15, 15, 64)        18496     
__________

In [0]:
def train_model(model, train_x, train_y, validate_x, validate_y):
  datagen = ImageDataGenerator(rotation_range = 0, width_shift_range = 0.1, height_shift_range = 0.1, fill_mode = 'nearest', horizontal_flip = True, 
                               vertical_flip = False)
  datagen.fit(train_x)
  for e in range(10):
    batches = 0
    for x_batch, y_batch in datagen.flow(train_x, train_y, batch_size = 40000):
      model.fit(x_batch, y_batch)
              
  return model.fit(train_x, train_y, batch_size=batch_size, epochs = epochs, verbose = 1, validation_data = (validate_x, validate_y))

In [0]:
def test_model(x_test,y_test):
  
  score = model.evaluate(x_test,y_test)
  print('test loss = ', score[0])
  print('test accuracy = ', score[1]*100)

In [0]:
def plot_training_graph(training_history, foldNumber):
  plotaccuracy = plt.plot(range(1,epochs+1), training_history.history['acc'], range(1,epochs+1), training_history.history['val_acc'])
  subtitle = "fold number " +foldNumber
  plt.subtitle(subtitle)
  plt.xlabel('number of epochs')
  plt.ylabel('Accuracy')
  plt.legend(('Train Accuracy', 'Test Accuracy'))
  plt.show(plotaccuracy)
  
  print('validation loss:', training_history.history['val_acc'][-1] )
  print('validation accuracy:', training_history.history['val_acc'][-1] )
    

In [0]:
x_folds = np.array_split(x_train,5)
y_folds = np.array_split(y_train,5)

for i in range(5):
  
  train_x = list(x_folds)
  validate_x = train_x.pop(i)
  train_x = np.concatenate(train_x)
  
  train_y = list(y_folds)
  validate_y = train_y.pop(i)
  train_y = np.concatenate(train_y)
  
  print('no of samples in training set: ',len(train_x))
  print('no of samples in validation set: ',len(validate_x))
  
  input_shape = train_x.shape[1:]
  print('shape of images: ',input_shape)
  
  model = getModel(input_shape)
  
  test_model(x_test,y_test)
  
  
avg_validation_loss=sum(validation_loss)/len(validation_loss)
avg_validation_accuracy=sum(validation_accuracy)/len(validation_accuracy)

print('\nAverage validation loss = ', avg_validation_loss)
print('\nAverage validation Accuracy = ', avg_validation_accuracy)
  